In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

/home/dro/.local/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("train_dataset_train.csv")
test = pd.read_csv('test_dataset_test.csv')

/tmp/ipykernel_661513/398589533.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train_dataset_train.csv")
/tmp/ipykernel_661513/398589533.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('test_dataset_test.csv')


In [3]:
df['index_oper'] = df['index_oper'].astype(str).apply(lambda x: x if len(x) >= 4 else '0').astype(float).astype(int)
test['index_oper'] = test['index_oper'].astype(str).apply(lambda x: x if len(x) >= 4 else '0').astype(float).astype(int)

In [4]:
df["is_in_yandex"] = pd.Categorical(df["is_in_yandex"])
df["is_in_yandex"].astype('category').cat.codes
df["is_in_yandex"] = df["is_in_yandex"].cat.codes

In [5]:
test["is_in_yandex"] = pd.Categorical(test["is_in_yandex"])
test["is_in_yandex"].astype('category').cat.codes
test["is_in_yandex"] = test["is_in_yandex"].cat.codes

In [6]:
df["is_return"] = pd.Categorical(df["is_return"])
df["is_return"].astype('category').cat.codes
df["is_return"] = df["is_return"].cat.codes

In [7]:
test["is_return"] = pd.Categorical(test["is_return"])
test["is_return"].astype('category').cat.codes
test["is_return"] = test["is_return"].cat.codes

Обьединим список не нужных строк с списком строк типа object

In [8]:
cols_to_enc = ['type', 'is_privatecategory']

In [9]:
from sklearn.preprocessing import LabelEncoder

for col in cols_to_enc:
    enc = LabelEncoder()
    enc.fit(df[col])
    
    df[col] = enc.transform(df[col])
    test[col] = enc.transform(test[col])

In [10]:
col_obj = df.select_dtypes(include=['object']).columns.values
col_obj = list(set(col_obj) ^ set(["id", "label"]))

In [11]:
col_obj

['oper_type + oper_attr', 'label', 'name_mfi', 'id']

In [12]:
X = df.drop(col_obj, axis = 1)
y = df[["label"]]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

## Обучение модели

In [14]:
from sklearn.metrics import recall_score

def scorer(y_true, y_pred):

    return 'recall', recall_score(y_true, y_pred > 0.01, average='macro'), True

In [15]:
import lightgbm as lgb

In [16]:
clf = lgb.LGBMClassifier(num_leaves=12023, random_state=42, n_estimators=100)

In [17]:
clf.fit(X_train, y_train, eval_set=(X_test, y_test), eval_metric=scorer)

/home/dro/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dro/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	valid_0's binary_logloss: 0.106419	valid_0's recall: 0.5
[2]	valid_0's binary_logloss: 0.100707	valid_0's recall: 0.5
[3]	valid_0's binary_logloss: 0.0965881	valid_0's recall: 0.5
[4]	valid_0's binary_logloss: 0.0934473	valid_0's recall: 0.5
[5]	valid_0's binary_logloss: 0.0908926	valid_0's recall: 0.5
[6]	valid_0's binary_logloss: 0.0887974	valid_0's recall: 0.5
[7]	valid_0's binary_logloss: 0.0870255	valid_0's recall: 0.5
[8]	valid_0's binary_logloss: 0.0855371	valid_0's recall: 0.5
[9]	valid_0's binary_logloss: 0.0842832	valid_0's recall: 0.5
[10]	valid_0's binary_logloss: 0.0832354	valid_0's recall: 0.5
[11]	valid_0's binary_logloss: 0.0823079	valid_0's recall: 0.825031
[12]	valid_0's binary_logloss: 0.0815044	valid_0's recall: 0.832284
[13]	valid_0's binary_logloss: 0.0807774	valid_0's recall: 0.837681
[14]	valid_0's binary_logloss: 0.0801407	valid_0's recall: 0.851904
[15]	valid_0's binary_logloss: 0.0796503	valid_0's recall: 0.86266
[16]	valid_0's binary_logloss: 0.0791581	v

LGBMClassifier(num_leaves=12023, random_state=42)

## Оценка точности

In [18]:
pred = clf.predict_proba(X_test)[:, 1] > 0.007

In [19]:
from sklearn.metrics import recall_score

score = recall_score(y_test, pred, average = "macro" )

In [20]:
print("Recall", score)

Recall 0.8809808493631688


In [21]:
ss = pd.read_csv('sample_solution.csv')

In [22]:
ss['label'] = (clf.predict_proba(test[X_test.columns])[:, 1] > 0.007).astype(int)


In [23]:
ss.to_csv('solut.csv', index=False)

In [24]:
ss['label'].mean()

0.19133825